In [66]:
sc

In [67]:
import pandas as pd
from pyspark.mllib.recommendation import *
import random
from operator import *

In [68]:
data = pd.read_table("./train_triplets.txt", header=None, nrows=1000)
data.columns = ['user', 'song', 'counts']
data.head()
# print data.user.unique().shape
# print data.song.unique().shape
# print sum(data.counts)

,user,song,counts
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [89]:
userID = pd.DataFrame({'user':data.user.unique(), 'uid':range(data.user.unique().shape[0])})
songID = pd.DataFrame({'song':data.song.unique(), 'sid':range(data.song.unique().shape[0])})
print userID.shape
print songID.shape

(13, 2)
(951, 2)


In [70]:
data2 = data.copy()
data2 = pd.merge(data2, userID, on='user')
data2 = pd.merge(data2, songID, on='song')

In [71]:
data2.to_csv("./id_train_triplets.csv", index=False, header=False)

# sc

In [72]:
musicData = sc.textFile('./id_train_triplets.csv', use_unicode=False, ).map(lambda x: x.split(',')).map(lambda x: (int(x[3]), int(x[4]), int(x[2])))

In [73]:
musicData.map

[(0, 0, 1), (0, 1, 1)]

In [74]:
trainData, validationData, testData = musicData.randomSplit([0.4, 0.4, 0.2], 13)
trainData.cache()
validationData.cache()
testData.cache()

PythonRDD[8370] at RDD at PythonRDD.scala:48

In [107]:
def modelEval(model,dataset,rank):
    
    datasetMap = dataset.map(lambda x:(x[0],x[1])).groupByKey().collectAsMap()
    trainDataMap = trainData.map(lambda x:(x[0],x[1])).groupByKey().collectAsMap() # collectAsMap: list[(a,1)..] - > dict{a:1..}
    allSongs = musicData.map(lambda x:(x[1])).collect()
    
    total=0.0
    userCount=0
    
    for user in datasetMap.keys(): # validation data: datasetMap
        songsInTrainData=trainDataMap.get(user) #not dict[key]: -> dictionary.get("bogus")  # <-- No default specified -- defaults to None
        songsNotInTrainData=[]
        for x in allSongs:
            if x not in songsInTrainData:
                songsNotInTrainData.append(x)
        result=[]
        for x in songsNotInTrainData:
            record=(user,x)
            result.append(record)
        finalRDD=sc.parallelize(result)
        finalResult=model.predictAll(finalRDD) # what does finalResult look like??? score? rank? (user, songNotInTrain, ???score|rank)
        prediction = finalResult.map(lambda x: (x[2], x[1])).sortByKey(False).map(lambda x: x[1]) #Asecending=False ->Desc: High to Low
        # ordered list of Song (not in training data, songs which the user didn't listen to before): Desc: from the best recommendation
        
        trueSongs=datasetMap.get(user) # validation data: list of songs -> ready to compare with prediction of validation data
        X=len(trueSongs)
        
        
        total += len(set(prediction.take(X)).intersection(set(trueSongs)))/float(X)
        userCount += 1
        
    
    print "The model score for rank %d is %f"%(rank,float(total/float(userCount)))


In [143]:
trainData.take(3)

[(0, 1, 1), (0, 2, 2), (0, 6, 2)]

In [121]:
ranks=[10, 20, 30, 40]
for r in ranks:
    Model = ALS.trainImplicit(trainData, rank=r, seed=345) # 
    modelEval(Model, validationData, r)

The model score for rank 10 is 0.004617
The model score for rank 20 is 0.001296
The model score for rank 30 is 0.002161
The model score for rank 40 is 0.002161


In [122]:
ranks=[10, 20, 30, 40]
for r in ranks:
    Model = ALS.train(trainData, rank=r, seed=345)
    modelEval(Model, validationData, r)

The model score for rank 10 is 0.002161
The model score for rank 20 is 0.004195
The model score for rank 30 is 0.002593
The model score for rank 40 is 0.002161


In [123]:
bestModel_1 = ALS.trainImplicit(trainData, rank=10, seed=345) 
bestModel_2 = ALS.train(trainData, rank=20, seed=345) 
modelEval(bestModel, testData, rank=10)
modelEval(bestModel, testData, rank=20)

The model score for rank 10 is 0.001789
The model score for rank 20 is 0.001789


In [141]:
topFive_1 = bestModel_1.recommendProducts(1,5)
topFive_2 = bestModel_2.recommendProducts(1,5)
for x in topFive_1:
    print x
print '\n'
for x in topFive_2:
    print x

Rating(user=1, product=112, rating=0.9824783192851354)
Rating(user=1, product=124, rating=0.9824783192851354)
Rating(user=1, product=120, rating=0.9824783192851354)
Rating(user=1, product=123, rating=0.9822629804740902)
Rating(user=1, product=115, rating=0.9822629804740902)


Rating(user=1, product=197, rating=6.738464191373322)
Rating(user=1, product=823, rating=5.611994807301073)
Rating(user=1, product=715, rating=5.611994807301073)
Rating(user=1, product=632, rating=5.300217219601269)
Rating(user=1, product=940, rating=4.98843937137643)


In [155]:
musicData.map(lambda x: x[1]).distinct()

PythonRDD[21270] at RDD at PythonRDD.scala:48

In [154]:
# SongMap=musicData.map(lambda x: x[1]).distinct().collectAsMap()

# for i, song in enumerate(topFive_1):
#     print "Artist " + str(i) + ": " + artistMap.get(artist[1])